In [1]:
import csv
import pandas as pd
import numpy as np
import os
import torch
import pickle
import datetime
from torch.utils.data import TensorDataset, DataLoader
from imblearn.over_sampling import RandomOverSampler # conda install -c conda-forge imbalanced-learn
from sklearn.model_selection import train_test_split

In [2]:
! ls data

lvl2_train.pkl	Ys_train.pkl


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [11]:
def pick_easy_tasks(model, x, mask, y, N, K, epoch):
    criterion = nn.BCELoss(reduction='none')
    model.train(False)
    y_hat = model(x, mask)

    # Pick tasks in the batch for which the loss is less than 1 / N.
    loss = torch.add((1 / GAMMA) * criterion(y_hat, y), -1 / N)
    easy_indices = list((loss < 0).nonzero()) if epoch >= K else list(loss.nonzero())

    easy_timeseries = torch.empty((len(easy_indices), x.size()[1], x.size()[2]))
    easy_masks = torch.empty((len(easy_indices), x.size()[1], x.size()[2]))
    easy_labels = torch.empty((len(easy_indices),))

    for i, index in enumerate(easy_indices):
        index = index.item()
        easy_timeseries[i] = x[index]
        easy_masks[i] = mask[index]
        easy_labels[i] = y[index]

    print(f"samples picked: {len(easy_indices)}, positive samples picked: {sum(easy_labels)}, K: {K}, epoch: {epoch}")

    model.train(True)
    return torch.Tensor(easy_timeseries).to(device), torch.Tensor(easy_masks).to(device), torch.Tensor(easy_labels).to(device)

In [12]:
with open("data/lvl2_train.pkl", "rb") as f:
    data = pickle.load(f).sort_values(['subject_id', 'hadm_id', 'icustay_id'])

with open("data/Ys_train.pkl", "rb") as f:
    labels = pickle.load(f).sort_values(['subject_id', 'hadm_id', 'icustay_id'])
    
data = data.reset_index(drop=True).droplevel(level="LEVEL2", axis=1)
features = data["mean"].to_numpy()
masks = data["mask"].to_numpy()

features = np.split(features, [48 * i for i in range(1, len(features) // 48)])
masks = np.split(masks, [48 * i for i in range(1, len(masks) // 48)])

features = torch.tensor(features, dtype=torch.float).to(device)
masks = torch.tensor(masks, dtype=torch.float).to(device)
labels = torch.squeeze(torch.tensor(labels.to_numpy(), dtype=torch.float).to(device))

print(f"Shape of features: {features.size()}")
print(f"Shape of masks: {masks.size()}")
print(f"Shape of labels: {labels.size()}")
assert len(features) == len(labels)
assert features.size() == masks.size()

Shape of features: torch.Size([15591, 48, 104])
Shape of masks: torch.Size([15591, 48, 104])
Shape of labels: torch.Size([15591])


In [13]:
X = list(zip(features, masks))

In [14]:
ros = RandomOverSampler(random_state=0)
X_resampled, labels_resampled = ros.fit_resample(X, labels)

/home/iandre3/.conda/envs/mimic-extract-2/lib/python3.9/site-packages/sklearn/utils/validation.py:746: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = np.asarray(array, order=order, dtype=dtype)
/home/iandre3/.conda/envs/mimic-extract-2/lib/python3.9/site-packages/sklearn/utils/validation.py:746: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = np.asarray

In [15]:
features_resampled, masks_resampled = list(zip(*X_resampled))

features_resampled = np.array([np.array(xi) for xi in features_resampled])
masks_resampled = np.array([np.array(xi) for xi in masks_resampled])

print(features_resampled.shape)
print(masks_resampled.shape)
print(labels_resampled.shape)

features = torch.tensor(features_resampled, dtype=torch.float).to(device)
masks = torch.tensor(masks_resampled, dtype=torch.float).to(device)
labels = torch.tensor(labels_resampled, dtype=torch.float).to(device)

print(features.shape)
print(masks.shape)
print(labels.shape)

(28482, 48, 104)
(28482, 48, 104)
(28482,)
torch.Size([28482, 48, 104])
torch.Size([28482, 48, 104])
torch.Size([28482])


In [16]:
train_features, test_features, train_masks, test_masks, train_labels, test_labels = train_test_split(features, masks, labels, test_size=0.1, random_state=42)
train_features, validation_features, train_masks, validation_masks, train_labels, validation_labels = train_test_split(train_features, train_masks, train_labels, test_size=0.1, random_state=42)

def create_easy_train_dataloader(model, N, K, epoch):
    easy_timeseries, easy_masks, easy_labels = pick_easy_tasks(model, train_features, train_masks, train_labels, N, K, epoch)
    easy_train_dataset = TensorDataset(easy_timeseries, easy_masks, easy_labels)
    if len(easy_train_dataset) == 0:
        return None, len(train_features)
    return DataLoader(easy_train_dataset, batch_size=32, shuffle=True), len(train_features)

test_dataset = TensorDataset(test_features, test_masks, test_labels)
validation_dataset = TensorDataset(validation_features, validation_masks, validation_labels)

test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)
validation_dataloader = DataLoader(validation_dataset, batch_size=len(validation_dataset), shuffle=False)

print(f"Train dataset size: {len(train_features)} ICU stays with 48-hour timeseries for {features.size()[2]} measurements each")
print(f"Test dataset size: {len(test_dataset)} ICU stays with 48-hour timeseries for {features.size()[2]} measurements each")
print(f"Validation dataset size: {len(validation_dataset)} ICU stays with 48-hour timeseries for {features.size()[2]} measurements each")

Train dataset size: 23069 ICU stays with 48-hour timeseries for 104 measurements each
Test dataset size: 2849 ICU stays with 48-hour timeseries for 104 measurements each
Validation dataset size: 2564 ICU stays with 48-hour timeseries for 104 measurements each


In [17]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score

constant_classifier = DummyClassifier(strategy="constant", constant=1)
uniform_classifier = DummyClassifier(strategy="uniform")

inputs, labels = None, None
for inp, mask, lab in validation_dataloader:
    inputs, labels = inp, lab
    labels = labels.clone().cpu().detach()

constant_classifier.fit([0] * len(labels), labels)
uniform_classifier.fit([0] * len(labels), labels)
y_constant = constant_classifier.predict([0] * len(labels))
y_uniform = uniform_classifier.predict([0] * len(labels))

print(f"Baseline F1 score constant 1: {f1_score(labels, y_constant)}")
print(f"Baseline F1 score uniform: {f1_score(labels, y_uniform)}")

Baseline F1 score constant 1: 0.6694343539180073
Baseline F1 score uniform: 0.48359375


In [26]:
GAMMA = 0.5
N = 16
LAMBDA = 1.3
LR = 0.001
OMEGA = 0.001
K = 1
TOLERANCE = 0.005

In [27]:
import torch.nn as nn
import torch.nn.utils.rnn as rnn

class TinyModel(torch.nn.Module):
    def __init__(self):
        super(TinyModel, self).__init__()
        self.rnn = nn.GRU(input_size=104, hidden_size=32, num_layers=1, bidirectional=False, batch_first=True)
        self.fc1 = nn.Linear(in_features=32, out_features=1)

    def forward(self, x, mask):
        try:
            x = x * mask
        except:
            print(x, mask)
        _, h_n = self.rnn(x)
        fc1_out = self.fc1(torch.squeeze(h_n))
        # Multiply prediction value before sigmoid activation by gamma.
        # This is part of the micro framework.
        r = torch.mul(fc1_out, GAMMA)
        res = torch.sigmoid(r)
        return torch.squeeze(res)

tinymodel = TinyModel()
tinymodel.to(device)

print('The model:')
print(tinymodel)

The model:
TinyModel(
  (rnn): GRU(104, 32, batch_first=True)
  (fc1): Linear(in_features=32, out_features=1, bias=True)
)


In [28]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(tinymodel.parameters(), LR)

def loss_func(y_hat, y, model):
    # Add a dimension when there is only one item in the batch.     
    if len(y_hat.size()) == 0:
        y_hat = torch.unsqueeze(y_hat, dim=0)

    l1 = OMEGA * sum(p.abs().sum() for p in model.parameters())
    cr = (1 / GAMMA) * criterion(y_hat, y)
    
    # Calculate the loss. The regularization is just an l1 regularization with a coefficient.
    # The criterion is the binary cross-entropy multiplied by 1 / GAMMA, a part of the micro framework.
    return l1 + cr

    # The paper also says to subtract batch_size / N, but this value is different for different batches depending
    # on the number of easy tasks in the batch, making it hard to compare loss across batches. Crucially, this 
    # is confusing when you look at the loss on training data (non-full batches) vs loss on testing data (full-size batches).
    # This makes me think that actually the paper meant that you first pick easy tasks across the entire testing set,
    # and then split it into batches rather than split into batches first and pick easy tasks after.
#     batch_size = y_hat.size()[0]    
#     neg = batch_size / N
#     return l1 + cr - neg

In [29]:
previous_loss = 1e10

for epoch in range(100):
    print(f"\n\nN is set to {N}")
    running_loss = 0.0
    number_examples_used = 0
    dataloader, max_num_examples = create_easy_train_dataloader(tinymodel, N, K, epoch)
    
    if dataloader is not None:
        for i, (features, masks, labels) in enumerate(dataloader):
            tinymodel.train(True)
            optimizer.zero_grad()

            # Train on easy sequences.
            output = tinymodel(features, masks)
            loss = loss_func(output, labels, tinymodel)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            number_examples_used += len(features)
            if (i + 1) % 50 == 0:
                print(f'Epoch: [{epoch + 1}, training batch {i + 1:5d}] running training loss: {running_loss / 50:.3f}')
                running_loss = 0.0
    
    running_test_loss = 0.0
    tinymodel.train(False)
    print(f"In epoch {epoch + 1}, number of examples trained on is {number_examples_used} out of {max_num_examples}")
    for i, (test_inputs, test_masks, test_labels) in enumerate(test_dataloader):
        test_outputs = tinymodel(test_inputs, test_masks)
        test_loss = loss_func(test_outputs, test_labels, tinymodel)
        running_test_loss += test_loss
    
    normalized_running_test_loss = running_test_loss / len(test_dataloader)
    print(f"Test loss: {normalized_running_test_loss}")
    
    if previous_loss - normalized_running_test_loss < TOLERANCE and epoch > 20:
        print(f"Early stopping, as {previous_loss - normalized_running_test_loss} < {TOLERANCE}")
        break
    
    previous_loss = normalized_running_test_loss
    
    # Once we have passed K warm-up epochs, decrease N by a factor of lambd
    # and as such increase 1 / N, so increase the loss threshold that defines
    # what tasks are considrered easy.
    if epoch >= K:
        N = N / LAMBDA

print("Finished Training")

N is set to 16


samples picked: 23069, positive samples picked: 11510.0, K: 1, epoch: 0
Epoch: [1, training batch    50] running training loss: 2.268
Epoch: [1, training batch   100] running training loss: 1.768
Epoch: [1, training batch   150] running training loss: 1.446
Epoch: [1, training batch   200] running training loss: 1.206
Epoch: [1, training batch   250] running training loss: 1.130
Epoch: [1, training batch   300] running training loss: 1.098
Epoch: [1, training batch   350] running training loss: 1.073
Epoch: [1, training batch   400] running training loss: 1.103
Epoch: [1, training batch   450] running training loss: 1.108
Epoch: [1, training batch   500] running training loss: 1.073
Epoch: [1, training batch   550] running training loss: 1.026
Epoch: [1, training batch   600] running training loss: 1.069
Epoch: [1, training batch   650] running training loss: 1.059
Epoch: [1, training batch   700] running training loss: 1.035
In epoch 1, number of examples trained on i

In [30]:
from sklearn.metrics import f1_score, roc_auc_score

tinymodel.train(False)
out = None
lab = None
for inputs, masks, labels in validation_dataloader:
    outputs = tinymodel(inputs, masks)
    out = outputs
    outputs = outputs > 0.5
    lab = labels
    labels = labels.detach().cpu().numpy()
    outputs = outputs.detach().cpu().numpy()
    print(f"F1 score: {f1_score(labels, outputs)}")
    print(f"ROC AUC: {roc_auc_score(labels, outputs)}")

F1 score: 0.8370745836350469
ROC AUC: 0.8240431771993233


In [4]:
import matplotlib.pyplot as plt
from sklearn import metrics

# Fill with real values

def mean(x):
    return sum(x) / len(x)

coverages = []
calculated_metrics = []
    
# criterion = nn.BCELoss(reduction='none')
# loss = (1 / GAMMA) * criterion(out, lab)
# ordered_indices = torch.argsort(loss)
# ordered_outputs = 1.0 * (torch.index_select(out, 0, ordered_indices))
# ordered_labels = torch.index_select(lab, 0, ordered_indices)

# ordered_outputs = ordered_outputs.cpu().detach().numpy()
# ordered_labels = ordered_labels.cpu().detach().numpy()

AUC = 0

# for coverage in np.arange(0, 1.005, 0.01):
#     length = len(ordered_outputs)
#     cutoff_index = int(length * coverage) + 1
    
#     included_outputs = ordered_outputs[0:cutoff_index]
#     included_labels = ordered_labels[0:cutoff_index]
    
#     if sum(included_labels) == 0 or sum(included_outputs) == 0 or sum(included_labels) == cutoff_index or sum(included_outputs) == cutoff_index:
#         print(coverage)
#         continue

#     metric = metrics.roc_auc_score(included_labels, included_outputs)
#     calculated_metrics.append(metric)
#     coverages.append(coverage)
    
#     AUC += metric * 0.01
    
    
confidences = 0.5 + abs(out - 0.5)
ordered_indices = torch.argsort(confidences, descending=True)
ordered_outputs = torch.index_select(out, 0, ordered_indices)
ordered_labels = torch.index_select(lab, 0, ordered_indices)

for confidence_threshold in np.arange(0.99, 0.01, -0.01):
    above_confidence_threshold_indices = torch.nonzero(ordered_outputs[ordered_outputs > confidence_threshold]).squeeze()
    above_confidence_outputs = torch.index_select(ordered_outputs, 0, above_confidence_threshold_indices)
    above_confidence_labels = torch.index_select(ordered_labels, 0, above_confidence_threshold_indices)
    
    metric = metrics.roc_auc_score(above_confidence_labels.cpu().detach().numpy(), above_confidence_outputs.cpu().detach().numpy())
    calculated_metrics.append(metric)
    coverage = len(above_confidence_outputs) / len(ordered_outputs)
    coverages.append(coverage)
    
    AUC += metric * 0.01
    
fig, ax = plt.subplots()
ax.set_linewidths([3])
ax.plot(coverages, calculated_metrics, linewidth=3)
ax.set_title('Metric-Coverage')
ax.set_xlabel('Coverage')
ax.set_ylabel('AUC')
plt.show()
print(f"Area under the metric-coverage plot: {AUC}")

NameError: name 'out' is not defined